In [ ]:
# import libraries
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
from tensorflow.keras.layers import TextVectorization
import matplotlib.pyplot as plt
print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

# Load the datasets
train_df = pd.read_csv("train-data.tsv", sep='\t', header=None, names=['label', 'text'])
test_df = pd.read_csv("valid-data.tsv", sep='\t', header=None, names=['label', 'text'])

# Map labels to integers: ham=0, spam=1
label_map = {'ham': 0, 'spam': 1}
train_df['label'] = train_df['label'].map(label_map)
test_df['label'] = test_df['label'].map(label_map)

In [ ]:
# Extract text and labels
# Clean empty strings
train_df = train_df[train_df['text'].str.strip().astype(bool)]
test_df = test_df[test_df['text'].str.strip().astype(bool)]

train_texts = train_df['text'].values
train_labels = train_df['label'].values.astype(int)
test_texts = test_df['text'].values
test_labels = test_df['label'].values.astype(int)


# Set hyperparameters for vectorization
max_vocab_size = 10000
max_sequence_length = 100

vectorizer = TextVectorization(max_tokens=max_vocab_size, output_mode='int', output_sequence_length=max_sequence_length)
vectorizer.adapt(train_texts)

model = keras.Sequential([
    vectorizer,
    keras.layers.Embedding(input_dim=max_vocab_size, output_dim=64),  # removed mask_zero=True
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3), loss='binary_crossentropy', metrics=['accuracy'])

model.fit(train_texts, train_labels, epochs=10, validation_data=(test_texts, test_labels))

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    inputs = tf.constant([pred_text])
    spam_prob = float(model.predict(inputs)[0][0])
    label = "spam" if spam_prob > 0.5 else "ham"
    return [spam_prob, label]

pred_text = "how are you doing today?"
prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
